## Logger

In [1]:
import logging

class MainLogger:
    def __init__(self):
        self.formatter = logging.Formatter('[%(levelname)s] %(message)s')
        self.logger = self._get_logger()

    def _set_handler(self):
        handler = logging.StreamHandler()
        handler.setLevel(logging.DEBUG)
        handler.setFormatter(self.formatter)

        return handler 
    
    def _get_logger(self):
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.DEBUG)
        logger.addHandler(self._set_handler())
    
        return logger
    
    def debug(self, message):
        self.logger.debug(message)

    def info(self, message):
        self.logger.info(message)

    def warning(self, message):
        self.logger.warning(message)

    def error(self, message):
        self.logger.error(message, exc_info=True)

logger = MainLogger()

## Google API Key 유효성 검사

In [2]:
import os
import json 
import requests
from dotenv import load_dotenv

load_dotenv()

def validate_google_api_key():
    """Google API Key 유효성 검사하는 함수"""
    key_name = "GOOGLE_API_KEY"
    if key_name not in os.environ:
        return f"{key_name} 정보가 없습니다. 환경변수를 확인해주세요."
    
    result = requests.post(
        url= "https://generativelanguage.googleapis.com/v1/models/gemini-1.5-flash:generateContent",
        data=b'{"contents":[{"parts":[{"text":""}]}]}',
        headers={
            "Content-Type": "application/json",
            "x-goog-api-key": os.getenv(key_name)
        }
    )

    if result.status_code != 200:
        logger.debug(json.loads(result.content))
    
    logger.info("Google API Key validation succeeded.") # logger 대체

validate_google_api_key()

# API 틀릴 때 
# {'error': {
#         'code': 400,
#         'message': 'API key not valid. Please pass a valid API key.',
#         'status': 'INVALID_ARGUMENT',
#         'details': [
#             {
#                 '@type': 'type.googleapis.com/google.rpc.ErrorInfo',
#                 'reason': 'API_KEY_INVALID',
#                 'domain': 'googleapis.com',
#                 'metadata': {'service': 'generativelanguage.googleapis.com'}
#             }
#         ]
#     }
# }

[INFO] Google API Key validation succeeded.


## Gemini 모듈

In [3]:
import os 
import asyncio
from pathlib import Path
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.output_parsers import StrOutputParser

class Gemini:
    def __init__(
        self,
        user_info=None,
        character_info=None,
        chat_history=None
    ) -> None:
        
        # 입력값에 대한 변수
        self.inputs = self._get_inputs(user_info, character_info, chat_history)
        
        # 체인에 대한 변수
        self.template_path = "./static/templates/Demo.prompt"
        self.model_name = "gemini-1.5-flash"
        self.temperature = 0.7
        self.chain = self._make_chain()

    def _get_inputs(self, user_info, character_info, chat_history):
        """input_data들을 하나의 딕셔너리로 바꾸는 메서드"""
        inputs = {
            "user_info": user_info,
            "character_info" : character_info,
            "chat_history": self._wrap_message(chat_history)
        }

        return inputs

    def _wrap_message(self, chat_history):
        """chat_history에 Message 객체 씌우는 메서드"""
        if not chat_history:
            return []

        chat_messages = []
        for log in chat_history:
            if log["role"] == "user":
                chat = HumanMessage(log["content"])
            else:
                chat = AIMessage(log["content"])
            
            chat_messages.append(chat)
        
        return chat_messages
    
    @staticmethod
    def read_template(filepath:str) -> str:
        """프롬프트 파일을 읽고 텍스트로 반환하는 함수

        Args:
            filepath (str): markdown 파일 경로

        Returns:
            str: markdown 파일에서 추출된 텍스트
        """
        file = Path(filepath)
        
        if not file.is_file():
            file_text = f"[ERROR] 파일 경로를 찾을 수 없습니다.(INPUT PATH: {filepath})"
        else:
            file_text = file.read_text(encoding="utf-8")

        return file_text
    
    def _get_prompts(self):
        """프롬프트 객체 만드는 메서드"""
        # 프롬프트 설정
        template = self.read_template(self.template_path)
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", template),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
            ]
        )

        return prompt
    
    def _make_chain(self):
        """Chain 만드는 메서드"""
        # 프롬프트 설정
        prompt = self._get_prompts()

        # 모델 설정
        model = ChatGoogleGenerativeAI(
            model=self.model_name,
            temperature=self.temperature
        )

        # 출력 파서 설정
        output_parser = StrOutputParser()

        # 체인 만들기
        chain = prompt | model | output_parser

        return chain
        
    # 방안1, 2에 적용했으나 쓰지 않을 예정
    async def astream(self, input):
        self.inputs["input"] = input

        output = ""
        result = self.chain.astream(self.inputs)
        async for token in result:
            output += token 
            # 한글자씩 스트리밍
            for char in token:
                print(char, end="", flush=True)
            
        
        return output
    
    async def astream_yield(self, input):
        self.inputs["input"] = input

        result = self.chain.astream(self.inputs)
        async for token in result:
            # 한글자씩 스트리밍
            for char in token:
                await asyncio.sleep(0.01)
                yield char

    # NEW
    def add_history(self, input, output):
        self.inputs["chat_history"].extend(
        [
            HumanMessage(content=input),
            AIMessage(content=output)
        ]
    )


## 입력값 형태

In [4]:
user_info = {
    "user_name": "",
    "user_birthdate": "",
    "user_gender": ""
}

character_info = {
    "character_name": "",
    "character_gender": "",
    "character_personality": "",
    "character_details": "",
    "relation_type": ""
}

chat_history = [
    {"role":"user", "content":"hello"},
    {"role":"character", "content":"Hi"},
    {"role":"user", "content":"I'm so happy"}
]

In [5]:
gemini = Gemini(
        user_info = user_info,
        character_info = character_info, 
        chat_history = chat_history
    )

output = await gemini.astream("나 오늘 우울해서 빵을 샀어")

빵은 언제나 좋은 선택이지! 맛있는 빵으로 기분이 나아지길 바라! 😊 무슨 빵을 샀는지 말해줄 수 있어? 혹시 좋아하는 빵집이 있다면 추천해줄 수도 있고! 😊 


## 방안 1

In [6]:
user_info = {
    "user_name": "",
    "user_birthdate": "",
    "user_gender": ""
}

character_info = {
    "character_name": "",
    "character_gender": "",
    "character_personality": "",
    "character_details": "",
    "relation_type": ""
}

chat_history = [
    {"role":"user", "content":"hello"},
    {"role":"character", "content":"Hi"},
    {"role":"user", "content":"I'm so happy"}
]

In [7]:
# 방안 1
async def main(input):
    gemini = Gemini(
        user_info = user_info,
        character_info = character_info, 
        chat_history = chat_history
    )

    output = await gemini.astream(input)
    # DB 대신 임시 추가
    chat_history.extend(
        [
            {"role": "user", "content":input},
            {"role": "character", "content": output}
        ]
    )

    return output 

In [8]:
import nest_asyncio
nest_asyncio.apply()
import asyncio 

output = asyncio.run(main("나 오늘 우울해서 빵을 샀어"))
print("-"*100)
print(output)

아, 그래요? 빵을 먹으면 기분이 좀 나아지시길 바라요. 어떤 빵을 사셨어요? 혹시 좋아하는 빵집이 있으신가요? 빵 이야기는 언제나 즐거워요! 😊 
----------------------------------------------------------------------------------------------------
아, 그래요? 빵을 먹으면 기분이 좀 나아지시길 바라요. 어떤 빵을 사셨어요? 혹시 좋아하는 빵집이 있으신가요? 빵 이야기는 언제나 즐거워요! 😊 



In [9]:
output = asyncio.run(main("단팥빵을 샀어"))
print("-"*100)
print(output)

단팥빵! 팥 좋아하시는군요! 달콤하고 부드러운 단팥빵은 뭔가 위로가 되는 ----------------------------------------------------------------------------------------------------
단팥빵! 팥 좋아하시는군요! 달콤하고 부드러운 단팥빵은 뭔가 위로가 되는 


In [10]:
output = asyncio.run(main("내가 무슨 빵 샀다고 했지?"))
print("-"*100)
print(output)

죄송해요! 제가 집중력이 떨어졌나 봐요. 😅  다시 한번 말씀해주시겠어요? 무슨 빵을 사셨는지 궁금해요! 

----------------------------------------------------------------------------------------------------
죄송해요! 제가 집중력이 떨어졌나 봐요. 😅  다시 한번 말씀해주시겠어요? 무슨 빵을 사셨는지 궁금해요! 




## 방안 2

In [11]:
user_info = {
    "user_name": "",
    "user_birthdate": "",
    "user_gender": ""
}

character_info = {
    "character_name": "",
    "character_gender": "",
    "character_personality": "",
    "character_details": "",
    "relation_type": ""
}

chat_history = [
    {"role":"user", "content":"hello"},
    {"role":"character", "content":"Hi"},
    {"role":"user", "content":"I'm so happy"}
]

In [12]:
# 방안 2
async def main(input):
    gemini = Gemini(
        user_info = user_info,
        character_info = character_info, 
        chat_history = chat_history
    )

    inputs = gemini.inputs
    chain = gemini.chain 

    inputs["input"] = input
    output = ""
    result = chain.astream(inputs)
    async for token in result:
        output += token 
        for text in token:
            # 웹소켓
            print(text, end="", flush=True)

    # DB 대신 임시 추가
    chat_history.extend(
        [
            {"role": "user", "content":input},
            {"role": "character", "content": output}
        ]
    )

    return output

In [13]:
import nest_asyncio
nest_asyncio.apply()
import asyncio 

output = asyncio.run(main("나 오늘 우울해서 빵을 샀어"))
print("-"*100)
print(output)

빵은 언제나 좋은 선택이지! 맛있는 빵 먹으면서 기분 풀어봐. 😊  무슨 빵을 샀는지 말해줄 수 있어? 혹시 나눠 먹을 수 있는 빵이라면... 😜  
----------------------------------------------------------------------------------------------------
빵은 언제나 좋은 선택이지! 맛있는 빵 먹으면서 기분 풀어봐. 😊  무슨 빵을 샀는지 말해줄 수 있어? 혹시 나눠 먹을 수 있는 빵이라면... 😜  



In [14]:
output = asyncio.run(main("단팥빵을 샀어"))
print("-"*100)
print(output)

단팥빵! 팥 좋아하는 사람으로써 정말 맛있겠다! 🤤  달콤한 팥 앙금이랑 쫄깃한 빵 껍질의 조합은 최고지.  혹시 따뜻하게 데워서 먹었어? 따뜻하게 먹으면 더 맛있을 것 같아. 😊  

혹시 팥 좋아하는 사람이라면 팥빙수도 추천해! 팥빙수 먹으면 시원하고 달콤해서 기분이 좋아질 거야. 🍧 
----------------------------------------------------------------------------------------------------
단팥빵! 팥 좋아하는 사람으로써 정말 맛있겠다! 🤤  달콤한 팥 앙금이랑 쫄깃한 빵 껍질의 조합은 최고지.  혹시 따뜻하게 데워서 먹었어? 따뜻하게 먹으면 더 맛있을 것 같아. 😊  

혹시 팥 좋아하는 사람이라면 팥빙수도 추천해! 팥빙수 먹으면 시원하고 달콤해서 기분이 좋아질 거야. 🍧 



In [15]:
output = asyncio.run(main("내가 무슨 빵 샀다고 했지?"))
print("-"*100)
print(output)

앗, 미안해! 😅  내가 너무 팥빙수 생각에 휩쓸려서 깜빡했네.  네가 무슨 빵 샀는지 기억 안 나!  다시 한번 말해줄 수 있니? 😊 
----------------------------------------------------------------------------------------------------
앗, 미안해! 😅  내가 너무 팥빙수 생각에 휩쓸려서 깜빡했네.  네가 무슨 빵 샀는지 기억 안 나!  다시 한번 말해줄 수 있니? 😊 



## 🚨 방안1, 방안2 출력 시 버벅거림 발생

## ⭐(확정) 방안 3

In [16]:
user_info = {
    "user_name": "",
    "user_birthdate": "",
    "user_gender": ""
}

character_info = {
    "character_name": "",
    "character_gender": "",
    "character_personality": "",
    "character_details": "",
    "relation_type": ""
}

chat_history = [
    {"role":"user", "content":"hello"},
    {"role":"character", "content":"Hi"},
    {"role":"user", "content":"I'm so happy"}
]

In [17]:
# 웹 소켓 적용
async def main(input):
    gemini = Gemini(
        user_info = user_info,
        character_info = character_info, 
        chat_history = chat_history
    )

    output = ""
    async for char in gemini.astream_yield(input):
        output += char
        print(char, end="", flush=True)
        
    # DB 대신 임시 추가
    chat_history.extend(
        [
            {"role": "user", "content":input},
            {"role": "character", "content": output}
        ]
    )

In [18]:
import nest_asyncio
nest_asyncio.apply()
import asyncio 

asyncio.run(main("나 오늘 우울해서 빵을 샀어"))

아, 그래요? 빵은 언제나 좋은 선택이죠! 맛있는 빵을 먹으면 기분이 좀 나아지시겠어요. 😊 어떤 빵을 사셨는지 궁금하네요! 혹시 좋아하는 빵 종류가 있으신가요? 


In [19]:
asyncio.run(main("단팥빵을 샀어"))

단팥빵! 팥 좋아하는 사람으로서 정말 맛있겠네요. 😋 달콤한 팥 앙금이 가득한 단팥빵은 뭔가 위로가 되는 맛이죠.  혹시 따뜻하게 데워 드시는 건 어떠세요? 뜨끈한 단팥빵은 더욱 맛있을 것 같아요. 

오늘 우울하셨다니, 단팥빵 드시면서 기분이 좀 나아지셨으면 좋겠어요. 😊 


In [20]:
asyncio.run(main("내가 무슨 빵 샀다고 했지?"))

앗, 죄송해요! 제가 깜빡했네요. 😅  단팥빵이라고 말씀하셨죠? 

단팥빵 맛있게 드시고 기분이 나아지셨으면 좋겠어요! 😊  혹시 빵 드시면서 뭐 하고 싶으신가요? 



## 🚨 너무 빨리 입력하면 중간에 출력되다 마는 현상 발생 -> model을 계속 불러와서 그런건 아닐까 추정 (방안 4 적용)

## 방안 4

In [21]:
user_info = {
    "user_name": "",
    "user_birthdate": "",
    "user_gender": ""
}

character_info = {
    "character_name": "",
    "character_gender": "",
    "character_personality": "",
    "character_details": "",
    "relation_type": ""
}

chat_history = [
    {"role":"user", "content":"hello"},
    {"role":"character", "content":"Hi"},
    {"role":"user", "content":"I'm so happy"}
]

In [22]:
# 웹 소켓 적용
gemini = Gemini(
    user_info = user_info,
    character_info = character_info, 
    chat_history = chat_history
)

async def main(input):
    output = ""
    async for char in gemini.astream_yield(input):
        output += char
        print(char, end="", flush=True)
        
    # DB 대신 임시 추가
    gemini.inputs["chat_history"].extend(
        [
            HumanMessage(content=input),
            AIMessage(content=output)
        ]
    )

In [23]:
import nest_asyncio
nest_asyncio.apply()
import asyncio 

asyncio.run(main("나 오늘 우울해서 빵을 샀어"))

빵을 먹으면 기분이 나아지나요? 어떤 빵을 사셨어요? 맛있게 드시고 기분이 좀 나아지셨으면 좋겠네요! 😊 혹시 힘든 일이 있으면 털어놓고 싶으시면 언제든지 말씀해주세요. 



In [24]:
asyncio.run(main("단팥빵을 샀어"))

단팥빵! 달콤한 팥 앙금이 가득한 단팥빵은 정말 맛있죠! 😋  단팥빵 먹으면서 달콤한 기분으로 우울함이 싹 사라지셨으면 좋겠어요. 😊  혹시 힘든 일이 있으면 언제든지 말씀해주세요.  

혹시 단팥빵과 함께 먹으면 더 맛있는 음료는 없으신가요? 😊 


In [25]:
asyncio.run(main("내가 무슨 빵 샀다고 했지?"))

아, 죄송해요! 제가 깜빡했네요. 😅  단팥빵이 아니라 다른 빵을 사셨죠? 어떤 빵을 사셨는지 다시 말씀해주시겠어요? 😊 

혹시 빵 이름을 말씀해주시면 제가 빵에 대한 정보를 찾아서 같이 이야기 나눠볼 수도 있을 것 같아요! 🍞 




## add history 추가 테스트

In [26]:
user_info = {
    "user_name": "",
    "user_birthdate": "",
    "user_gender": ""
}

character_info = {
    "character_name": "",
    "character_gender": "",
    "character_personality": "",
    "character_details": "",
    "relation_type": ""
}

chat_history = [
    {"role":"user", "content":"hello"},
    {"role":"character", "content":"Hi"},
    {"role":"user", "content":"I'm so happy"}
]

In [33]:
# 웹 소켓 적용
gemini = Gemini(
    user_info = user_info,
    character_info = character_info, 
    chat_history = chat_history
)

async def main(input):
    output = ""
    async for char in gemini.astream_yield(input):
        output += char
        print(char, end="", flush=True)
        
    # DB 에 input, output 넣는 코드 자리
    # history 속성에 새로운 대화 추가
    gemini.add_history(input, output)

In [28]:
import nest_asyncio
nest_asyncio.apply()
import asyncio 

asyncio.run(main("나 오늘 우울해서 빵을 샀어"))

빵을 먹으면 기분이 나아지길 바라요! 맛있는 빵을 골랐는지 궁금해요. 어떤 빵을 샀는지 말해줄 수 있나요? 😊 


In [29]:
asyncio.run(main("단팥빵을 샀어"))

단팥빵! 달콤하고 부드러운 팥앙금이 가득한 단팥빵은 정말 맛있죠!  따뜻

In [30]:
asyncio.run(main("내가 무슨 빵 샀다고 했지?"))

맞아요! 단팥빵이라고 말씀하셨죠! 😊  달콤한 팥앙금이 가득한 단팥빵은 정말 맛있어요. 

혹시 어떤 브랜드의 단팥빵인지 궁금해요? 


In [31]:
# 웹소켓 예시코드
# 참고 사이트: https://github.com/zhiyuan8/FastAPI-websocket-tutorial/blob/main/fastapi-chatbot/main.py
# @app.websocket("/ws")
# async def websocket_endpoint(websocket: WebSocket):
#     """Websocket endpoint for real-time AI responses."""
#     await websocket.accept()
#     while True:
        
#         user_message = await websocket.receive_text()
#         async for ai_response in get_ai_response(user_message):
#             await websocket.send_text(ai_response)

In [32]:
# 예제 
import asyncio

async def stream_tokens():
    tokens = ["나는", "오늘", "점심을", "먹었다"]
    for token in tokens:
        for char in token:
            await asyncio.sleep(0.1)  # 실제 네트워크 지연을 시뮬레이션
            yield char

async def main():
    async for char in stream_tokens():
        print(char, end="", flush=True)
    print()  # 마지막에 줄바꿈

asyncio.run(main())

나는오늘점심을먹었다
